In [43]:
import nltk
import pandas as pd
import re
import string

from gensim.models.word2vec import Word2Vec
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, roc_auc_score
from urllib.request import urlopen

# Part 1 - Working with Text Data

### Use Python string methods remove irregular whitespace from the following string:

In [2]:
whitespace_string = "\n\n  This is a    string   that has  \n a lot of  extra \n   whitespace.   "

print(whitespace_string)



  This is a    string   that has  
 a lot of  extra 
   whitespace.   


In [3]:
whitespace_string = ' '.join(whitespace_string.split())
print(whitespace_string)

This is a string that has a lot of extra whitespace.


### Use Regular Expressions to take the dates in the following .txt file and put them into a dataframe with columns for:

[RegEx dates.txt](https://github.com/ryanleeallred/datasets/blob/master/dates.txt)

- Day
- Month
- Year


In [4]:
days = []
months = []
years = []

with urlopen('https://raw.githubusercontent.com/ryanleeallred/datasets/master/dates.txt') as file:
    
    encoding = file.info().get_param('charset', 'utf8')
    html = file.read().decode(encoding)

    data = re.sub(',','', html)
    data = data.splitlines()
    for line in data:
        split_str = re.findall(r"\S+", line)
        months.append(split_str[0])
        days.append(split_str[1])
        years.append(split_str[2])
    


In [5]:
df = pd.DataFrame()
df['Day'] = days
df['Month'] = months
df['Year'] = years

In [6]:
df.head()

,Day,Month,Year
0,8,March,2015
1,15,March,2015
2,22,March,2015
3,29,March,2015
4,5,April,2015


# Part 2 - Bag of Words 

### Use the twitter sentiment analysis dataset found at this link for the remainder of the Sprint Challenge:

[Twitter Sentiment Analysis Dataset](https://raw.githubusercontent.com/ryanleeallred/datasets/master/twitter_sentiment_binary.csv)

 ### Clean and tokenize the documents ensuring the following properties of the text:

1) Text should be lowercase.

2) Stopwords should be removed.

3) Punctuation should be removed.

4) Tweets should be tokenized at the word level. 

(The above don't necessarily need to be completed in that specific order.)

### Output some cleaned tweets so that we can see that you made all of the above changes.


In [31]:
df = pd.read_csv("https://raw.githubusercontent.com/ryanleeallred/datasets/master/twitter_sentiment_binary.csv") 

# we create a small utility function so we can apply our clean method to series/df
def regex_cleaner(df_cell):
    current = ''.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\ / \ / \S+)", "", df_cell))
    return current.lower()
                             
df['SentimentText'] = df['SentimentText'].apply(regex_cleaner)
df['tokenized'] = df['SentimentText'].apply(nltk.word_tokenize)

In [32]:
df.head()

,Sentiment,SentimentText,tokenized
0,0,is so sad for my apl friend,"[is, so, sad, for, my, apl, friend]"
1,0,i missed the new moon trailer,"[i, missed, the, new, moon, trailer]"
2,1,omg its already 730 o,"[omg, its, already, 730, o]"
3,0,omgaga im sooo im gunna cry ive be...,"[omgaga, im, sooo, im, gunna, cry, ive, been, ..."
4,0,i think mi bf is cheating on me tt,"[i, think, mi, bf, is, cheating, on, me, tt]"


### How should TF-IDF scores be interpreted? How are they calculated?

TF-IDF scores are a model for word importance in the context of it's document:corpus

#### TF-IDF is calculated by two contextual divisions that assign weights to words occuring in documents.
The first assumes that the greater the occurence of a word in a singular document, the greater it's importance. This is referred to as the Term Frequency and can be adjusted for many factors, such as document length in comparison to other documents.

The second assumes that the greater the occurence of a word in the corpus, the lesser it's importance and uses this assumption as an offset. This is referred to as the Inverse Document Frequency, and it reduces the words weight, or importance as occurence across the corpus increases and buffs the weight of rare terminology.


TF-IDF is calculated by multiplying Term Frequency (TF) and Inverse Document Frequency(IDF). Hence the name abbreviation :P 



# Part 3 - Document Classification

1) Use Train_Test_Split to create train and test datasets.

2) Vectorize the tokenized documents using your choice of vectorization method. 

 - Stretch goal: Use both of the methods that we talked about in class.

3) Create a vocabulary using the X_train dataset and transform both your X_train and X_test data using that vocabulary.

4) Use your choice of binary classification algorithm to train and evaluate your model's accuracy. Report both train and test accuracies.

 - Stretch goal: Use an error metric other than accuracy and implement/evaluate multiple classifiers.



In [34]:
X = df['SentimentText']
y = df['Sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, stratify=y)

stop_words = set(nltk.corpus.stopwords.words('english'))

In [39]:
logreg_vect_pipe = make_pipeline(CountVectorizer(stop_words=stop_words),
                        LogisticRegression(solver='lbfgs',
                                       max_iter=1000))
grid_params = [{'countvectorizer__ngram_range' : [(1, 1), (1, 2), (1,3), (1,4), (1,5)],
                  'countvectorizer__max_features' : [5, 10, 25, 50, 100, None]}]

lr_grid = GridSearchCV(logreg_vect_pipe, 
                       grid_params, 
                       cv=3,
                      scoring='roc_auc')
lr_grid.fit(X_train, y_train);

In [40]:
print ('Train data: ROC AUC: %.2f' % roc_auc_score(lr_grid.predict(X_train), y_train), 
       'Test data: ROC AUC: %.2f' % roc_auc_score(lr_grid.predict(X_test), y_test))

Train data: ROC AUC: 0.98 Test data: ROC AUC: 0.76


# Part 4 -  Word2Vec

1) Fit a Word2Vec model on your cleaned/tokenized twitter dataset. 

2) Display the 10 words that are most similar to the word "twitter"

In [45]:
vector_model = Word2Vec(df.tokenized)
vector_model.wv.most_similar('twitter')

[('facebook', 0.7222739458084106),
 ('myspace', 0.7012851238250732),
 ('everyone', 0.6537954807281494),
 ('list', 0.644665002822876),
 ('fb', 0.636321485042572),
 ('youtube', 0.6268463730812073),
 ('comment', 0.615507185459137),
 ('updates', 0.6137909293174744),
 ('blog', 0.613542377948761),
 ('comments', 0.605195164680481)]